<a href="https://colab.research.google.com/github/mchu-1/bio/blob/main/bio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Assembly Dependencies

In [ ]:
!pip install biopython dnacauldron dnachisel dnaweaver -q

# Import Assembly Dependencies

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Restriction import *
import dnacauldron as dnc
import dnachisel as dch

# Define Golden Gate Site

In [ ]:
# Replace degenerate bases in Type IIS RS
def define_gg_site(enzyme):
  """
  Replace degenerate bases with random GC bases for T2S RS.
  """
  B = []
  for x in enzyme.elucidate().split("^")[0]:
    if "N" in x:
      B.append(dch.random_dna_sequence(length=1, gc_share=1, seed=0))
    else:
      B.append(x)
  return "".join(B)

# Get Golden Gate Part

In [ ]:
# Get sequence between Golden Gate handles
def get_gg_part(enzyme, seq, overhang: int = 4):
  # Default overhang length is 4 nt.
  gg_part = enzyme.catalyse(seq)[1] + enzyme.catalyse(seq)[-1][:overhang]
  return gg_part

# Switch Golden Gate Handles

In [ ]:
# Switch Golden Gate handles
def switch_gg_handles(enzyme_1, enzyme_2, seq, overhang: int = 4):
  """
  Switch T2S enzyme handles for a given sequence.
  Default overhang length is 4 nt.
  """
  seq_ = get_gg_part(enzyme_1, seq, overhang)
  new_handle = Seq(define_gg_site(enzyme_2))
  new_seq = new_handle + seq_ + new_handle.reverse_complement()

  return Seq(new_seq)

# Generate Golden Gate Switching Primers

In [ ]:
# Generate primers to switch Golden Gate handles
def get_switching_handles(enzyme_1, enzyme_2, seq, overhang: int = 4):
  handle = Seq("GGCTAC") # Flanking bases for RE binding
  new_seq = switch_gg_handles(enzyme_1, enzyme_2, seq, overhang)
  new_seq_F = Seq(handle + enzyme_2.catalyse(new_seq)[0] + enzyme_2.catalyse(new_seq)[1][:20])
  new_seq_R = Seq(enzyme_2.catalyse(new_seq)[1][-20:] + enzyme_2.catalyse(new_seq)[-1] + handle).reverse_complement()
  return new_seq_F, new_seq_R